In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
# csv가 홀수개가 있어야 함 
# (다수결로 정하는데 동률이 나오면 안되므로)

# ensemble할 csv 파일이 모여있는 폴더 위치
# 그 폴더에 다른 용도의 csv 파일이 있으면 안됨
CSV_ROOT = "./make_ensemble"

# 이미지 크기
img_size = (2048, 2048)

# ensemble한 csv 파일을 저장할 위치와 파일 이름
SAVED_DIR = "./ensemble.csv"

In [3]:
def mask_to_rle(mask):

    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_to_mask(rle, height, width):
    s = rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(height * width, dtype=np.uint8)
    
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    
    return img.reshape(height, width)

In [4]:
print(CSV_ROOT)
csv_list = []
for file in os.listdir(CSV_ROOT):
    if file[-4:] == ".csv":
        print(file)
        csv_dir = CSV_ROOT +"/"+ file
        csv_list.append(csv_dir)

print(".")
print(".")

csv_num = len(csv_list)

df = pd.DataFrame()
for csv in csv_list:
    df_tmp = pd.read_csv(csv)
    df_tmp["csv"] = csv
    df = pd.concat([df, df_tmp], ignore_index=True)

IMAGE = df['image_name'].unique()
CLASS = df['class'].unique()


image_name_list = []
class_list = []
rle_list = []

with tqdm(total=len(IMAGE)) as pbar:
    for ima in IMAGE:
        for cla in CLASS:
            mask = np.zeros(img_size[0]*img_size[1], dtype=np.uint8)
            mask = mask.reshape(img_size[0], img_size[1])
            for csv in csv_list:
                rle_part = df[(df['image_name'] == ima) & (df['class'] == cla) & (df['csv'] == csv)]['rle'].values[0]

                if str(type(rle_part)) !=  "<class 'str'>":
                    continue
                mask_part = rle_to_mask(rle_part, img_size[0], img_size[1])
                mask += mask_part
            
            mask = mask // (csv_num//2 +1)
            rle = mask_to_rle(mask)

            image_name_list.append(ima)
            class_list.append(cla)
            rle_list.append(rle)
        pbar.update(1)

print(".")
ensemble_df = pd.DataFrame({"image_name": image_name_list, "class": class_list, "rle": rle_list,})
print(".")
ensemble_df.to_csv(SAVED_DIR , index=False)
print('Ensemble Done...!')

./make_ensemble


FileNotFoundError: [Errno 2] No such file or directory: './make_ensemble'